**Ιωάννης Καπετανγεώργης**

**Αριθμός μητρώου: 1115201800061** 

Πριν από την εκτέλεση οποιουδήποτε από τα δύο μοντέλα  πρέπει να εκτελεστεί το παρακάτω κελί έτσι ώστε να γίνουν import όλα όσα χρησιμοποιούνται σε αυτά.

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import string
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
!pip install tweet-preprocessor
import preprocessor as p
!pip install emoji --upgrade
import emoji

Στο παρακάτω κελί υπάρχουν τα path των train set και validation set από όπου διαβάζονται τα δεδομένα.

Για την αντικατάσταση του validation set από το test set κατά την διόρθωση της εργασίας αρκεί να αλλάξει το path του validation set.
Δηλαδή αλλάζουμε την μεταβλητή validation_set_location αντικαθιστόντας το υπάρχον path με το αντίστοιχο path του test set.

In [ ]:
train_set_location = r'vaccine_train_set.csv' 
validation_set_location = r'vaccine_validation_set.csv'

# Μοντέλο 1 - Word Embeddings / Feed Forward Neural Network

In [ ]:
# download golve files
!wget https://nlp.stanford.edu/data/glove.twitter.27B.zip
!unzip glove.twitter.27B.zip

In [ ]:
trainSet = pd.read_csv(train_set_location,index_col=0)  # read csv file and store it to a dataframe
validationSet = pd.read_csv(validation_set_location,index_col=0)  # read csv file and store it to a dataframe

EMBEDDING_SIZE = 200
glove_file = "glove.twitter.27B."+str(EMBEDDING_SIZE)+'d.txt' # glove file that will be used

CLEAN_ULR_MENTIONS_NUMBERS = True   # remove urls, mentions and numbers from tweets
CLEAN_STOPWORDS = True # remove stopwords from tweets
CLEAN_EMOJIS = True # convert emojis to the equivelent text
CLEAN_PUNCUATIONS = True  # remove punctuation from tweets
CONVERT_TO_LOWERCASE = True # convert all text to lowercase
LEMMATIZATION = True # apply lemmatization to the tweets
STEMMING = False   # apply stemming to the tweets


def construct_embedding_dict(glove_file): # create an embedding dictionary using the glove file
    # every word is a key in the dictionary and has it's embedding vector as a value
    embedding_dict = {} # create empty dict
    count = 0
    with open(glove_file,'r') as f: #open the file
        for line in f:  # for every line of the file
          count += 1
          values=line.split() # split the line 
          word = values[0]  # the fist token is the word 
          vector = np.asarray(values[1:], 'float32')  # all the other tokens are the embedding vector of the word, so store them in an np array
          if vector.size == EMBEDDING_SIZE: # if the embedding vector has the wanted size
            embedding_dict[word] = vector # add word/embedding vector to the dictionary
    return embedding_dict

#create the embedding dictionary from the glove file  
embedding_dict =  construct_embedding_dict(glove_file)

p.set_options(p.OPT.URL,p.OPT.MENTION,p.OPT.NUMBER) # set tweet-preprocessor to remove urls, mentions and numbers

def clean_tweets(data,column):  # function that removes urls, mentions and numbers from all the tweets of the given dataframe
  for i,v in enumerate(data[column]): # for every tweet
      data.loc[i,column] = p.clean(v)
  return data

tweet_tokenizer = TweetTokenizer()   # initialize tweet tokenizer

stop_words = stopwords.words('english') # get english stop words

punctuations = list(string.punctuation) # get puncutations 
punctuations = punctuations + ['–','::','“','’','”','‘','`','...','``']  # add some extra puncutations

def removeEmojis(text): # function that converts emojis to the equivalent text
    text = emoji.demojize(text) # remove emojis
    return str(text) 

def removePuncutation(text):   # remove punctuation from the given text
    text = text.replace("-","")
    splitted = tweet_tokenizer.tokenize(text) # split the tweet into tokens
    new_text = []
    for word in splitted:  # for every token
      if word not in punctuations:    # keep it only if it is not a punctuation
        new_text.append(word)
    text = ' '.join(new_text)
    return text

def removeStopWordsAndPuncs(text):  # remove stop-words and punctuation from the given text
    text = text.replace("-","")
    text = text.replace("::"," ")
    splitted = word_tokenize(text)  # split the tweet into tokens
    new_text = []
    for word in splitted:   # for every token
      if word not in stop_words and word not in punctuations: # keep it only if it is not a punctuation or stop_word
        new_text.append(word)
    text = ' '.join(new_text)
    return text

def clean(text):    # clean the given tweet
    if(CONVERT_TO_LOWERCASE): # if it is enabled
      text = text.lower()  # convert all letters in to lowercase
    if(CLEAN_EMOJIS):
      text = removeEmojis(text)  # convert all emojis to the equivelent text
    if(CLEAN_STOPWORDS and CLEAN_PUNCUATIONS):
      text = removeStopWordsAndPuncs(text)    # remove all punctuation and stopwords
    elif(CLEAN_PUNCUATIONS):
      text = removePuncutation(text)    # remove all punctuation 
    return text

def cleanText(data,column):  # apply clean function to every tweet of the given dataframe
  data[column] = data[column].apply(clean)
  return data

# stemming
ps = SnowballStemmer("english") # intialize stemmer

def stemmTweet(text): # apply stemming to the given tweet
  return ' '.join([(ps.stem) for w \
                       in w_tokenizer.tokenize((text))])  # split the tweet into tokens and apply stemming to every token

def stem_data(data,column): # apply stemming into every tweet of the given dataframe
  data[column] = data[column].apply(stemmTweet)
  return data

lemmatizer = nltk.stem.WordNetLemmatizer()  # initialize Lemmatizer
w_tokenizer =  TweetTokenizer() # initialize tokenizer
 
def lemmatize_text(text): # apply lemmatization to the given tweet
  return ' '.join([(lemmatizer.lemmatize(w)) for w \
                       in w_tokenizer.tokenize((text))])  # split the tweet into tokens and apply lemmatization to every token

def lemmatize_data(data,column): # apply lemmatization into every tweet of the given dataframe
  data[column] = data[column].apply(lemmatize_text)
  return data   

if(CLEAN_ULR_MENTIONS_NUMBERS):
  trainSet = clean_tweets(trainSet,'tweet') # remove urls, mentions, numbers from all the tweets in the train set
  validationSet = clean_tweets(validationSet,'tweet') # remove urls, mentions, numbers from all the tweets in the validation set
if(CLEAN_EMOJIS or CLEAN_STOPWORDS or CLEAN_PUNCUATIONS or CONVERT_TO_LOWERCASE):
  trainSet = cleanText(trainSet,'tweet')    # clean the tweets of the train set 
  validationSet = cleanText(validationSet,'tweet')    # clean the tweets of the validation set 
if(LEMMATIZATION):
  trainSet = lemmatize_data(trainSet,'tweet') # apply lemmatization to all the tweets of the train set
  validationSet = lemmatize_data(validationSet,'tweet')  # apply lemmatization to all the tweets of the validation set
if(STEMMING):
  trainSet = stem_data(trainSet,'tweet')   # apply stemming to all the tweets of the train set
  validationSet = stem_data(validationSet,'tweet')   # apply stemming to all the tweets of the validation set


X_train = list(trainSet['tweet']) # convert the tweets of the train set into a list
y_train = list(trainSet['label']) # convert the labels of the train set into a list

X_validation = list(validationSet['tweet']) # convert the tweets of the validation set into a list
y_validation = list(validationSet['label']) # convert the labels of the validation set into a list

X_train_tokens = [sent.lower().split() for sent in X_train] # split every tweet of the train set into tokens
X_validation_tokens = [sent.lower().split() for sent in X_validation] # split every tweet of the validation set into tokens

def findAverageVector(words, model, num_features): # find the average vector of the given tweet
    feature_vec = np.zeros((num_features),dtype="float32")  # initialize
    wordCount = 0.

    for word in words:  # for every word of the text
      try:
        word_feature = model[word] # check the embedding dictionary to find the embedding vector of this word
      except KeyError:  # if the word is not in the disctionary
        continue
      else:
        if word_feature is not None: # if dictionary  contains that word
            wordCount = wordCount + 1.
            feature_vec = np.add(feature_vec,word_feature) # add the vector of this word to the total vector of the tweet
    if wordCount > 0:
      feature_vec = np.divide(feature_vec, wordCount) # find the average of the total vector of the tweet
    return feature_vec


def findTextsFeatureVectors(texts, model, num_features): # find the average feature vector for every text
    counter = 0
    textsFeatureVectors = np.zeros((len(texts),num_features), dtype='float32')  # initialize
    
    for text in texts:  # for every tweet
        textsFeatureVectors[counter] = findAverageVector(text, model, num_features) # find average vector of the tweet
        counter = counter + 1
    return textsFeatureVectors

# find feature vectors of train set
trainDataVecs = findTextsFeatureVectors(X_train_tokens, embedding_dict, EMBEDDING_SIZE)
# find feature vectors of validation set
validationDataVecs  = findTextsFeatureVectors(X_validation_tokens, embedding_dict, EMBEDDING_SIZE)

# convert train tweets and labels into tensors
x_train_tensor = torch.tensor(trainDataVecs, dtype=torch.float)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
# convert validation tweets and labels into tensors
x_validation_tensor = torch.tensor(validationDataVecs, dtype=torch.float)
y_validation_tensor = torch.tensor(y_validation, dtype=torch.long)




class Net(nn.Module):
    def __init__(self, D_in, H1, H2, H3,D_out):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()
        self.model_stack =  nn.Sequential(
                      nn.Linear(D_in, H1),        # input layer             
                      nn.ReLU(),  # activation function
                      nn.Dropout(0.5),  # dropout
                      nn.Linear(H1, H2),    # hidden layer 1
                      nn.ReLU(),  # activation function
                      nn.Dropout(0.25),  # dropout
                      nn.Linear(H2, H3),     # hidden layer 2
                      nn.ReLU(),  # activation function
                      nn.Dropout(0.1),  # dropout
                      nn.Linear(H3, D_out),   # output layer
                      # softmax is not needed after ouput layer because we are using nn.CrossEntropyLoss as the loss function
                      )


    def forward(self, x):
        x = self.flatten(x) # flatten the input in order to pass it to the sewuentian model
        logits = self.model_stack(x)
        return logits

#Define layer sizes
D_in = x_train_tensor.shape[1]
H1 = 64
H2 = 32
H3 = 16
D_out = 3

#define epochs
TRAINING_EPOCHS = 100

#Define Hyperparameters
learning_rate = 0.0025

#Initialize model, loss, optimizer
nn_model = Net(D_in, H1, H2, H3,D_out)
loss_func = nn.CrossEntropyLoss()
nn_optimizer = torch.optim.SGD(nn_model.parameters(), lr=learning_rate,momentum=0.5)

#Initialize dataloader for train set
train_dataset = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
#Initialize dataloader for validation set
validation_dataset = torch.utils.data.TensorDataset(x_validation_tensor, y_validation_tensor)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=16, shuffle=True)

def train_model(model,criterion,train_loader,validation_loader,optimizer,epochs = 200):
  train_loss = [] # store the train loss for every epoch
  valid_loss = []  # store the validation loss for every epoch
  for epoch in range(epochs):
    train_batch_losses = [] # store the loss of every batch of the train set
    validation_batch_losses = [] # store the loss of every batch of the validation set
    model.train() # set the model to training mode
    y_total_predict_train = []   # store the predictions of the the train set
    y_total_train = []     # store the labels of the the train set
    for x_batch, y_batch in train_loader:
      #Delete previously stored gradients
      optimizer.zero_grad()
      z = model(x_batch)
      loss = criterion(z, y_batch)  # compute the train loss
      train_batch_losses.append(loss.data.item()) # store the train loss of the current batch
      #Perform backpropagation starting from the loss calculated in this epoch
      loss.backward()
      #Update model's weights based on the gradients calculated during backprop
      optimizer.step()
      _, y_pred_tags = torch.max(z, dim = 1)  # get the prediction based on the maximum posibility
      y_total_predict_train = y_total_predict_train+ list(y_pred_tags) # store the predictions of the current batch
      y_total_train = y_total_train + (list(y_batch.detach().numpy())) # store the labels of the current batch
    correct = 0
    count = 0
    model.eval()   # set the model to evaluation mode
    y_total_valid = []   # store the predictions of the the validation set
    y_total_predict_valid = []      # store the labels of the the validation set
    for x_batch, y_batch in validation_loader:
      z = model(x_batch)
      loss = criterion(z, y_batch)   # compute the validation loss
      validation_batch_losses.append(loss.data.item())   # store the validation loss of the current batch
      _, label = torch.max(z,1)    # get the label prediction based on the maximum posibility
      correct += (label==y_batch).sum().item()    # compute how many corect preditions were made
      count += len(y_batch)
      y_total_predict_valid = y_total_predict_valid+ list(label) # store the predictions of the current batch
      y_total_valid= y_total_valid + list(y_batch.detach().numpy()) # store the labels of the current batch
    accuracy = 100*(correct/(count))  # compute validation accuracy based on the correct predictions
    current_train_loss = sum(train_batch_losses)/len(train_loader)  #compute the train loss of this epoch
    train_loss.append(current_train_loss)   # store the train loss in order to plot loss curves
    current_valid_loss = sum(validation_batch_losses)/len(validation_loader)   #compute the validation loss of this epoch
    valid_loss.append(current_valid_loss)   # store the validation loss in order to plot loss curves
    f1_score_train = f1_score(y_total_train,y_total_predict_train,average='weighted')# compure train f1-score
    f1_score_valid = f1_score(y_total_valid,y_total_predict_valid,average='weighted') # compure validation f1-score
    print(f"Epoch {epoch:3}: Train Loss = {current_train_loss:.5f} | Validation Loss = {current_valid_loss:.5f} | Accuracy = {accuracy:.4f} | Train-f1 = {f1_score_train:.4f} | Valid-F1 = {f1_score_valid:.4f}")
  return model,train_loss,valid_loss

# train the model
trained_model, train_loss,valid_loss = train_model(nn_model,loss_func,train_loader,validation_loader,nn_optimizer,TRAINING_EPOCHS)

def plot_loss_curves(t_loss,v_loss,epochs = 100):
  x = list(range(1,epochs+1))
  plt.plot(x, t_loss, 'r',label='Train Loss')
  plt.plot(x, v_loss, 'g',label='Validation Loss')
  plt.legend(loc='best')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.title('Loss vs Epoch curves')
  plt.show()

plot_loss_curves(train_loss,valid_loss,TRAINING_EPOCHS) 

y_pred = trained_model(x_validation_tensor)  # test the trained model on the validation set
y_pred_numpy = y_pred.detach().numpy()  # convert output to nupy array

# roc curve for classes
fpr = {}
tpr = {}
thresh ={}

n_class = 3  # number of classes/labels

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_validation_tensor, y_pred_numpy[:,i], pos_label=i)
    
# plotting the ROC curves
plt.plot(fpr[0], tpr[0], linestyle='--',color='orange', label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='Class 1 vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--',color='blue', label='Class 2 vs Rest')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('Multiclass ROC',dpi=300);  

_, y_pred_tags = torch.max(y_pred, dim = 1)# get tha labels of the predictions based on the label with the maximum possibility for each instance 
accuracy_countVec = accuracy_score(y_validation,y_pred_tags)  # accuracy metric
f1_score_countVec = f1_score(y_validation,y_pred_tags,average='weighted') # f1_score metric
print(" Accuracy: %.2f%%" %(accuracy_countVec*100))
print(" f1 score: %.2f%%" %(f1_score_countVec*100))
print(' Precision: %.2f%%' % (precision_score(y_validation, y_pred_tags,average='weighted')*100)) # precision
print(' Recall: %.2f%%' % (recall_score(y_validation, y_pred_tags,average='weighted')*100)) # recall
print(classification_report(y_validation, y_pred_tags))

# Μοντέλο 2 - TF-IDF / Feed Forward Neural Network

In [ ]:
CLEAN_ULR_MENTIONS_NUMBERS = True   # remove urls, mentions and numbers from tweets
CLEAN_STOPWORDS = True # remove stopwords from tweets
CLEAN_EMOJIS = True # convert emojis to the equivelent text
CLEAN_PUNCUATIONS = True  # remove punctuation from tweets
CONVERT_TO_LOWERCASE = True # convert all text to lowercase
LEMMATIZATION = True # apply lemmatization to the tweets
STEMMING = False   # apply stemming to the tweets

trainSet = pd.read_csv(train_set_location,index_col=0)  # read csv file and store it to a dataframe
validationSet = pd.read_csv(validation_set_location,index_col=0)  # read csv file and store it to a dataframe

p.set_options(p.OPT.URL,p.OPT.MENTION,p.OPT.NUMBER) # set tweet-preprocessor to remove urls, mentions and numbers

def clean_tweets(data,column):  # function that removes urls, mentions and numbers from all the tweets of the given dataframe
  for i,v in enumerate(data[column]): # for every tweet
      data.loc[i,column] = p.clean(v)
  return data

tweet_tokenizer = TweetTokenizer()   # initialize tweet tokenizer

stop_words = stopwords.words('english') # get english stop words

punctuations = list(string.punctuation) # get puncutations 
punctuations = punctuations + ['–','::','“','’','”','‘','`','...','``']  # add some extra puncutations
punctuations.remove('#')  # remove from puncutation
punctuations.remove('!')  # remove from puncutation
punctuations.remove('?')  # remove from puncutation 

def removeEmojis(text): # function that converts emojis to the equivalent text
    text = emoji.demojize(text) # remove emojis
    return str(text) 

def removePuncutation(text):   # remove punctuation from the given text
    text = text.replace("-","")
    text = text.replace("::"," ")
    splitted = word_tokenize(text) # split the tweet into tokens
    new_text = []
    for word in splitted:  # for every token
      if word not in punctuations:    # keep it only if it is not a punctuation
        new_text.append(word)
    text = ' '.join(new_text)
    return text

def removeStopWordsAndPuncs(text):  # remove stop-words and punctuation from the given text
    text = text.replace("-","")
    text = text.replace("::"," ")
    splitted = word_tokenize(text)  # split the tweet into tokens
    new_text = []
    for word in splitted:   # for every token
      if word not in stop_words and word not in punctuations: # keep it only if it is not a punctuation or stop_word
        new_text.append(word)
    text = ' '.join(new_text)
    return text

def clean(text):    # clean the given tweet
    if(CONVERT_TO_LOWERCASE): # if it is enabled
      text = text.lower()  # convert all letters in to lowercase
    if(CLEAN_EMOJIS):
      text = removeEmojis(text)  # convert all emojis to the equivelent text
    if(CLEAN_STOPWORDS and CLEAN_PUNCUATIONS):
      text = removeStopWordsAndPuncs(text)    # remove all punctuation and stopwords
    elif(CLEAN_PUNCUATIONS):
      text = removePuncutation(text)    # remove all punctuation 
    return text

def cleanText(data,column):  # apply clean function to every tweet of the given dataframe
  data[column] = data[column].apply(clean)
  return data

# stemming
ps = SnowballStemmer("english") # intialize stemmer

def stemmTweet(text): # apply stemming to the given tweet
  return ' '.join([(ps.stem) for w \
                       in w_tokenizer.tokenize((text))])  # split the tweet into tokens and apply stemming to every token

def stem_data(data,column): # apply stemming into every tweet of the given dataframe
  data[column] = data[column].apply(stemmTweet)
  return data

lemmatizer = nltk.stem.WordNetLemmatizer()  # initialize Lemmatizer
w_tokenizer =  TweetTokenizer() # initialize tokenizer
 
def lemmatize_text(text): # apply lemmatization to the given tweet
  return ' '.join([(lemmatizer.lemmatize(w)) for w \
                       in w_tokenizer.tokenize((text))])  # split the tweet into tokens and apply lemmatization to every token

def lemmatize_data(data,column): # apply lemmatization into every tweet of the given dataframe
  data[column] = data[column].apply(lemmatize_text)
  return data   

if(CLEAN_ULR_MENTIONS_NUMBERS):
  trainSet = clean_tweets(trainSet,'tweet') # remove urls, mentions, numbers from all the tweets in the train set
  validationSet = clean_tweets(validationSet,'tweet') # remove urls, mentions, numbers from all the tweets in the validation set
if(CLEAN_EMOJIS or CLEAN_STOPWORDS or CLEAN_PUNCUATIONS or CONVERT_TO_LOWERCASE):
  trainSet = cleanText(trainSet,'tweet')    # clean the tweets of the train set 
  validationSet = cleanText(validationSet,'tweet')    # clean the tweets of the validation set 
if(LEMMATIZATION):
  trainSet = lemmatize_data(trainSet,'tweet') # apply lemmatization to all the tweets of the train set
  validationSet = lemmatize_data(validationSet,'tweet')  # apply lemmatization to all the tweets of the validation set
if(STEMMING):
  trainSet = stem_data(trainSet,'tweet')   # apply stemming to all the tweets of the train set
  validationSet = stem_data(validationSet,'tweet')   # apply stemming to all the tweets of the validation set


X_train = list(trainSet['tweet']) # convert the tweets of the train set into a list
y_train = list(trainSet['label']) # convert the labels of the train set into a list

X_validation = list(validationSet['tweet']) # convert the tweets of the validation set into a list
y_validation = list(validationSet['label']) # convert the labels of the validation set into a list

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,3),max_features=300) #initialize the TfidfVectorizer
trainSet_transformed_tfidf = tfidf_vectorizer.fit_transform(X_train)  # run the vectorizer for all the tweets of the train set
validationSet_transformed_tfidf = tfidf_vectorizer.transform(X_validation)  # transform the tweets of the validation set

# convert train tweets and labels into tensors
x_train_tensor = torch.tensor(trainSet_transformed_tfidf.toarray(), dtype=torch.float)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
# convert validation tweets and labels into tensors
x_validation_tensor = torch.tensor(validationSet_transformed_tfidf.toarray(), dtype=torch.float)
y_validation_tensor = torch.tensor(y_validation, dtype=torch.long)



class Net(nn.Module):
    def __init__(self, D_in, H1, H2, H3,D_out):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()
        self.model_stack =  nn.Sequential(
                      nn.Linear(D_in, H1),    # input layer         
                      nn.ReLU(),  # activation function
                      nn.Dropout(0.5),  # dropout
                      nn.Linear(H1, H2),    # hidden layer 1
                      nn.ReLU(),  # activation function
                      nn.Dropout(0.25),  # dropout
                      nn.Linear(H2, H3),     # hidden layer 2
                      nn.ReLU(),  # activation function
                      nn.Dropout(0.1),  # dropout
                      nn.Linear(H3, D_out),   # output layer
                      # softmax is not needed after ouput layer because we are using nn.CrossEntropyLoss as the loss function
                      )


    def forward(self, x):
        x = self.flatten(x) # flatten the input in order to pass it to the sewuentian model
        logits = self.model_stack(x)
        return logits

#Define layer sizes
D_in = x_train_tensor.shape[1]
H1 = 20
H2 = 10
H3 = 10
D_out = 3

#define epochs
TRAINING_EPOCHS = 100

#Define Hyperparameters
learning_rate = 0.1

#Initialize model, loss, optimizer
nn_model = Net(D_in, H1, H2, H3,D_out)
loss_func = nn.CrossEntropyLoss()
nn_optimizer = torch.optim.SGD(nn_model.parameters(), lr=learning_rate)

#Initialize dataloader for train set
train_dataset = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
#Initialize dataloader for validation set
validation_dataset = torch.utils.data.TensorDataset(x_validation_tensor, y_validation_tensor)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=16, shuffle=True)

def train_model(model,criterion,train_loader,validation_loader,optimizer,epochs = 200):
  train_loss = [] # store the train loss for every epoch
  valid_loss = []  # store the validation loss for every epoch
  for epoch in range(epochs):
    train_batch_losses = [] # store the loss of every batch of the train set
    validation_batch_losses = [] # store the loss of every batch of the validation set
    model.train() # set the model to training mode
    y_total_predict_train = []   # store the predictions of the the train set
    y_total_train = []     # store the labels of the the train set
    for x_batch, y_batch in train_loader:
      #Delete previously stored gradients
      optimizer.zero_grad()
      z = model(x_batch)
      loss = criterion(z, y_batch)  # compute the train loss
      train_batch_losses.append(loss.data.item()) # store the train loss of the current batch
      #Perform backpropagation starting from the loss calculated in this epoch
      loss.backward()
      #Update model's weights based on the gradients calculated during backprop
      optimizer.step()
      _, y_pred_tags = torch.max(z, dim = 1)  # get the prediction based on the maximum posibility
      y_total_predict_train = y_total_predict_train+ list(y_pred_tags) # store the predictions of the current batch
      y_total_train = y_total_train + (list(y_batch.detach().numpy())) # store the labels of the current batch
    correct = 0
    count = 0
    model.eval()   # set the model to evaluation mode
    y_total_valid = []    # store the predictions of the the validation set
    y_total_predict_valid = []      # store the labels of the the validation set
    for x_batch, y_batch in validation_loader:
      z = model(x_batch)
      loss = criterion(z, y_batch)    # compute the validation loss
      validation_batch_losses.append(loss.data.item())   # store the validation loss of the current batch
      _, label = torch.max(z,1)     # get the label prediction based on the maximum posibility
      correct += (label==y_batch).sum().item()  # compute how many corect preditions were made
      count += len(y_batch)
      y_total_predict_valid = y_total_predict_valid+ list(label)   # store the predictions of the current batch
      y_total_valid= y_total_valid + list(y_batch.detach().numpy())  # store the labels of the current batch
    accuracy = 100*(correct/(count))  # compute validation accuracy based on the correct predictions
    current_train_loss = sum(train_batch_losses)/len(train_loader)  #compute the train loss of this epoch
    train_loss.append(current_train_loss)   # store the train loss in order to plot loss curves
    current_valid_loss = sum(validation_batch_losses)/len(validation_loader)    #compute the validation loss of this epoch
    valid_loss.append(current_valid_loss)   # store the validation loss in order to plot loss curves
    f1_score_train = f1_score(y_total_train,y_total_predict_train,average='weighted') # compure train f1-score
    f1_score_valid = f1_score(y_total_valid,y_total_predict_valid,average='weighted') # compure validation f1-score
    print(f"Epoch {epoch:3}: Train Loss = {current_train_loss:.5f} | Validation Loss = {current_valid_loss:.5f} | Accuracy = {accuracy:.4f} | Train-f1 = {f1_score_train:.4f} | Valid-F1 = {f1_score_valid:.4f}")
  return model,train_loss,valid_loss

# train the model
trained_model, train_loss,valid_loss = train_model(nn_model,loss_func,train_loader,validation_loader,nn_optimizer,TRAINING_EPOCHS)

def plot_loss_curves(t_loss,v_loss,epochs = 100):
  x = list(range(1,epochs+1))
  plt.plot(x, t_loss, 'r',label='Train Loss') # plotting t, a separately 
  plt.plot(x, v_loss, 'g',label='Validation Loss') # plotting t, b separately 
  plt.legend(loc='best')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.title('Loss vs Epoch curves')
  plt.show()

plot_loss_curves(train_loss,valid_loss,TRAINING_EPOCHS)

y_pred = trained_model(x_validation_tensor) # test the trained model on the validation set
y_pred_numpy = y_pred.detach().numpy() # convert output to nupy array

# roc curve for classes
fpr = {}
tpr = {}
thresh ={}

n_class = 3 # number of classes/labels

# compute roc curve values for every class/label
for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_validation_tensor, y_pred_numpy[:,i], pos_label=i)
    
# plotting  the ROC curves
plt.plot(fpr[0], tpr[0], linestyle='--',color='orange', label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='Class 1 vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--',color='blue', label='Class 2 vs Rest')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('Multiclass ROC',dpi=300);  

_, y_pred_tags = torch.max(y_pred, dim = 1) # get tha labels of the predictions based on the label with the maximum possibility for each instance 
accuracy_countVec = accuracy_score(y_validation,y_pred_tags)  # accuracy metric
f1_score_countVec = f1_score(y_validation,y_pred_tags,average='weighted') # f1_score metric
print(" Accuracy: %.2f%%" %(accuracy_countVec*100))
print(" f1 score: %.2f%%" %(f1_score_countVec*100))
print(' Precision: %.2f%%' % (precision_score(y_validation, y_pred_tags,average='weighted')*100)) # precision
print(' Recall: %.2f%%' % (recall_score(y_validation, y_pred_tags,average='weighted')*100)) # recall
print(classification_report(y_validation, y_pred_tags))